In [45]:
import requests
import collections
from operator import itemgetter
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime

In [46]:
font = '| font=Courier | size=14'
site = 'https://www.oddsshark.com'
lgs  = collections.defaultdict(dict)
lgs['UFC']['odds'] = requests.get(
    'https://io.oddsshark.com/ticker/ufc', 
    headers = {
        'referer': site
    }
)

for league, active in lgs['UFC']['odds'].json()['leagues'].items():
    lgs[league.upper()]['futures'] = BeautifulSoup(requests.get(f"{site}/{league}/odds/futures").content, 'html.parser')
    if active==True:
        lgs[league.upper()]['odds'] = requests.get(
            'https://io.oddsshark.com/ticker/' + league, 
            headers = {
                'referer': site
            }
        ).json()['matchups']

In [47]:
def simple_odds(odds):
    for game in odds:
        if game['type'] == 'date':
            print('--', game['date']['fullday'], game['date']['month'], game['date']['day'])
        if game["type"] == "matchup":
            if game['status'].startswith('FINAL'):
                print('----', game['status'], '| font=Courier | href=' + site + game['matchup_link'])
                print('----', game['away_short_name'], game['away_score'], '| font=Courier')
                print('----', game['home_short_name'], game['home_score'], '| font=Courier')
            else:
                print('----', game['status'], '| font=Courier | href=' + site + game['matchup_link'])
                print('----', game['away_short_name'], game['away_odds'] if game['away_odds'].startswith('-') else '+' + game['away_odds'], '| font=Courier')
                print('----', game['home_short_name'], game['home_odds'] if game['home_odds'].startswith('-') else '+' + game['home_odds'], '| font=Courier')

In [ ]:
lg_tks['UFC']

In [ ]:
print('UFC')
for fight in lg_tks['UFC']:
    if fight['type'] == 'event':
        print('--', fight['event'])
    if fight["type"] == 'matchup':
        if not fight['status']:
            print('----', fight['event_date'][11:])
            print('----', fight['away_name'], fight['away_odds'] if fight['away_odds'].startswith('-') else '+' + fight['away_odds'], '| font=Courier')
            print('----', fight['home_name'], fight['home_odds'] if fight['home_odds'].startswith('-') else '+' + fight['home_odds'], '| font=Courier')
        else:
            print('----', 'FINAL')
            print('----', fight['away_name'], fight['status'] if fight['away_name'] == fight['winner'] else '', '| font=Courier')
            print('----', fight['home_name'], fight['status'] if fight['home_name'] == fight['winner'] else '', '| font=Courier')

In [ ]:
print(soup.prettify())

In [48]:
def process_odds_section(soup, html_type, html_class):
    data = soup.find_all(html_type, class_=html_class)
    init_list = []
    daysofweek = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    for elem in data:
        if not elem.get_text().startswith(tuple(daysofweek)):
            init_list.append(elem.get_text().strip())
    return init_list

In [49]:
def create_futures_data(teams, opening, currents):
    
    futures_trios = []
    for a, b, c in zip(*[iter(currents)]*3):
        futures_trios.append([a,b,c])
    
    rows = []
    odds_inc = 0
    sports = ['MLB','NFL','NHL','NBA','College Basketball','College Football']
    for team in teams:
        if not team.startswith(tuple(sports)):
            rows.append(Future_Odds(future, team,
                         opening[odds_inc],
                         futures_trios[odds_inc][0],
                         futures_trios[odds_inc][1],
                         futures_trios[odds_inc][2]))
            odds_inc += 1
        else:
            rows.append([team])
            future = team
    return rows

In [50]:
def print_future_odds(sport):
    print('--', sport)
    opening_odds = process_odds_section(lg_futs[sport], 'div', ['op-item op-future-item', 'op-item op-future-item '])
    current_odds = process_odds_section(lg_futs[sport], 'div', 'op-item op-future-item no-vegas')
    team_list = process_odds_section(lg_futs[sport], ['div','span'], ['align-bottom',
                                                                      'op-team baseball op-odd',
                                                                      'op-team baseball op-even',
                                                                      'op-team basketball op-odd',
                                                                      'op-team basketball op-even',
                                                                      'op-team football op-odd',
                                                                      'op-team football op-even',
                                                                      'op-team hockey op-odd',
                                                                      'op-team hockey op-even'])
    all_odds = create_futures_data(team_list, opening_odds, current_odds)
    for odds in all_odds:
        if len(odds) > 1:
            print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format(odds[0], odds[1], odds[2], odds[3], odds[4], font))
        else:
            print('----', odds[0], font)
            print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format('Team','Opening','Bovada','BetOnline.AG','Sports Betting',font))

In [123]:
def print_all_odds(lgs):
    for sport in lgs:
        if sport != 'UFC':
            print(sport)
            if 'odds' in lgs[sport]:
                simple_odds(lgs[sport]['odds'])
        opening_odds = process_odds_section(lgs[sport]['futures'], 'div', ['op-item op-future-item', 'op-item op-future-item '])
        current_odds = process_odds_section(lgs[sport]['futures'], 'div', 'op-item op-future-item no-vegas')
        team_list = process_odds_section(lgs[sport]['futures'], ['div','span'], ['align-bottom',
                                                                          'op-team baseball op-odd',
                                                                          'op-team baseball op-even',
                                                                          'op-team basketball op-odd',
                                                                          'op-team basketball op-even',
                                                                          'op-team football op-odd',
                                                                          'op-team football op-even',
                                                                          'op-team hockey op-odd',
                                                                          'op-team hockey op-even'])
        all_odds = create_futures_data(team_list, opening_odds, current_odds)
        for odds in all_odds:
            if type(odds) is list:
                print('----', odds[0], font)
                print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format('Team','Opening','Bovada','BetOnline.AG','Sports Betting',font))
                futures = [fut for fut in all_odds if hasattr(fut, 'future') and fut.future == odds[0]]
                futures = sorted(futures, key=lambda x: sorting_provider(x, 'betonline'))
                for odds in futures:
                    print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format(odds.team, odds.opening, odds.bovada, odds.betonline, odds.sportsbetting, font))

In [73]:
opening_odds = process_odds_section(lgs['NHL']['futures'], 'div', ['op-item op-future-item', 'op-item op-future-item '])
current_odds = process_odds_section(lgs['NHL']['futures'], 'div', 'op-item op-future-item no-vegas')
team_list = process_odds_section(lgs['NHL']['futures'], ['div','span'], ['align-bottom',
                                                                          'op-team baseball op-odd',
                                                                          'op-team baseball op-even',
                                                                          'op-team basketball op-odd',
                                                                          'op-team basketball op-even',
                                                                          'op-team football op-odd',
                                                                          'op-team football op-even',
                                                                          'op-team hockey op-odd',
                                                                          'op-team hockey op-even'])
all_odds = create_futures_data(team_list, opening_odds, current_odds)

In [53]:
class Future_Odds:
    def __init__(self, future, team, opening, bovada, betonline, sportsbetting):
        self.future = future
        self.team = team
        self.opening = opening
        self.bovada = bovada
        self.betonline = betonline
        self.sportsbetting = sportsbetting
    def __repr__(self):
        return repr((self.future, self.team, self.opening, self.bovada, self.betonline, self.sportsbetting))

In [117]:
for odds in all_odds:
    if type(odds) is list:
        print('----', odds[0], font)
        print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format('Team','Opening','Bovada','BetOnline.AG','Sports Betting',font))
        futures = [fut for fut in all_odds if hasattr(fut, 'future') and fut.future == odds[0]]
        futures = sorted(futures, key=lambda x: sorting_provider(x, 'sportsbetting'))
        for odds in futures:
            print('------ {0:<21}{1:>8}{2:>8}{3:>14}{4:>15}{5}'.format(odds.team, odds.opening, odds.bovada, odds.betonline, odds.sportsbetting, font))

---- NHL - Stanley Cup 2020-21 | font=Courier | size=14
------ Team                  Opening  Bovada  BetOnline.AG Sports Betting| font=Courier | size=14
------ Tampa Bay                +700    +700          +600           +600| font=Courier | size=14
------ Colorado                 +700    +375          +700           +700| font=Courier | size=14
------ Vegas                    +700    +750          +700           +700| font=Courier | size=14
------ Toronto                 +2000    +750          +900           +900| font=Courier | size=14
------ NY Islanders            +2500   +1500         +1100          +1100| font=Courier | size=14
------ Carolina                +2500    +850         +1200          +1200| font=Courier | size=14
------ Boston                  +1200   +1100         +1400          +1400| font=Courier | size=14
------ Washington              +1600   +1400         +1400          +1400| font=Courier | size=14
------ Florida                 +5000   +2200         +1800    

In [120]:
def sorting_provider(x, provider):
    odd = getattr(x, provider)
    if odd == '':
        return float('inf')
    else:
        return int(odd)

In [63]:
sorting_provider('NHL - Stanley Cup 2020-21','NHL - Stanley Cup 2020-21')

0

In [112]:
whatvr = [futur for futur in all_odds if hasattr(futur, 'future') and futur.future == 'NHL - Stanley Cup 2020-21']
whatvr[0].bovada

'+150000'

In [99]:
whatvrsort = sorted(whatvr, key=lambda x: sorting_provider(x, 'bovada'))

AttributeError: 'Future_Odds' object has no attribute 'provider'

In [94]:
for odds in whatvrsort:
    print(odds.team, odds.bovada)

Colorado +375
Tampa Bay +700
Toronto +750
Vegas +750
Carolina +850
Boston +1100
Washington +1400
NY Islanders +1500
Edmonton +1800
Pittsburgh +2000
Florida +2200
Minnesota +2200
Winnipeg +2300
Montreal +2800
Dallas +3500
St. Louis +3500
Nashville +5000
Arizona +6600
NY Rangers +8000
Vancouver +10000
Calgary +15000
Chicago +15000
Los Angeles +15000
Philadelphia +20000
San Jose +20000
Columbus +100000
New Jersey +100000
Anaheim +150000
Buffalo +150000
Detroit +150000
Ottawa +150000


In [124]:
print_all_odds(lgs)

NHL
-- Thursday Apr 22
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/boston-buffalo-odds-april-22-2021-1367366
---- BOS 5 | font=Courier
---- BUF 1 | font=Courier
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/carolina-florida-odds-april-22-2021-1367371
---- CAR 4 | font=Courier
---- FLA 2 | font=Courier
---- FINAL SO | font=Courier | href=https://www.oddsshark.com/nhl/washington-new-york-odds-april-22-2021-1367376
---- WAS 1 | font=Courier
---- NYI 0 | font=Courier
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/new-jersey-pittsburgh-odds-april-22-2021-1367381
---- NJ 1 | font=Courier
---- PIT 5 | font=Courier
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/columbus-tampa-bay-odds-april-22-2021-1367386
---- CLB 1 | font=Courier
---- TB 3 | font=Courier
---- FINAL | font=Courier | href=https://www.oddsshark.com/nhl/philadelphia-new-york-odds-april-22-2021-1383936
---- PHI 3 | font=Courier
---- NYR 2 | font=Courier
---- FINA

---- MLB - World Series Championship 2021 | font=Courier | size=14
------ Team                  Opening  Bovada  BetOnline.AG Sports Betting| font=Courier | size=14
------ LA Dodgers               +400    +300          +300           +300| font=Courier | size=14
------ NY Yankees               +500    +700          +700           +700| font=Courier | size=14
------ San Diego                +900    +850          +950           +950| font=Courier | size=14
------ NY Mets                 +3000    +900         +1000          +1000| font=Courier | size=14
------ Chi White Sox           +1200   +1300         +1200          +1200| font=Courier | size=14
------ Atlanta                 +1100   +1400         +1600          +1600| font=Courier | size=14
------ Oakland                 +1800   +2000         +1600          +1600| font=Courier | size=14
------ Houston                 +1600   +2000         +2000          +2000| font=Courier | size=14
------ Milwaukee               +5000   +2500       

---- NBA - Championship 2021 | font=Courier | size=14
------ Team                  Opening  Bovada  BetOnline.AG Sports Betting| font=Courier | size=14
------ Brooklyn                +1400    +250          +225           +225| font=Courier | size=14
------ LA Lakers                +400    +325          +290           +290| font=Courier | size=14
------ LA Clippers              +450    +550          +575           +575| font=Courier | size=14
------ Utah                    +2500    +700          +850           +850| font=Courier | size=14
------ Milwaukee                +550   +1000          +900           +900| font=Courier | size=14
------ Philadelphia            +2500   +1000         +1200          +1200| font=Courier | size=14
------ Denver                  +1500   +3500         +2000          +2000| font=Courier | size=14
------ Miami                   +1300   +4000         +2800          +2800| font=Courier | size=14
------ Phoenix                 +6000   +1500         +2800      

---- College Football - National Championship 2021-22 | font=Courier | size=14
------ Team                  Opening  Bovada  BetOnline.AG Sports Betting| font=Courier | size=14
------ Alabama                  +280    +275          +260           +260| font=Courier | size=14
------ Clemson                  +300    +450          +475           +475| font=Courier | size=14
------ Ohio State               +500    +650          +650           +650| font=Courier | size=14
------ Georgia                  +800    +600          +700           +700| font=Courier | size=14
------ Oklahoma                 +800    +800          +800           +800| font=Courier | size=14
------ Texas A&M               +3000   +2500         +2500          +2500| font=Courier | size=14
------ Texas                   +4500   +3800         +2800          +2800| font=Courier | size=14
------ Florida                 +2500   +3300         +3300          +3300| font=Courier | size=14
------ Iowa State              +5000   

---- College Basketball - National Champion 2021-22 | font=Courier | size=14
------ Team                  Opening  Bovada  BetOnline.AG Sports Betting| font=Courier | size=14
------ Gonzaga                  +900    +550          +450           +450| font=Courier | size=14
------ Michigan                +1400   +1100         +1200          +1200| font=Courier | size=14
------ UCLA                    +1200   +1200         +1200          +1200| font=Courier | size=14
------ Villanova               +1400   +1200         +1200          +1200| font=Courier | size=14
------ Alabama                 +1600   +1500         +1600          +1600| font=Courier | size=14
------ Baylor                  +1200   +1800         +1800          +1800| font=Courier | size=14
------ Ohio State              +1400   +1800         +1800          +1800| font=Courier | size=14
------ Arkansas                +2000   +1800         +2000          +2000| font=Courier | size=14
------ Duke                    +1600   +1